# COVID-19 prediction for global and each country
#### Using regression analysis
#### Simple analysis for beginners

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

### Import data

In [ ]:
covid_19_data = pd.read_csv('/kaggle/input/novel-corona-virus-2019-dataset/covid_19_data.csv')
#open_line_list = pd.read_csv('/kaggle/input/novel-corona-virus-2019-dataset/COVID19_open_line_list.csv')

#confirmed = pd.read_csv('/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_confirmed.csv')
#deaths = pd.read_csv('/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_deaths.csv')
#recovered = pd.read_csv('/kaggle/input/novel-corona-virus-2019-dataset/time_series_covid_19_recovered.csv')
#line_list_data = pd.read_csv('/kaggle/input/novel-corona-virus-2019-dataset/COVID19_line_list_data.csv')

# 1. Global

## 1-1. Overview

In [ ]:
# Format and extract items

indi_date_list, confirmed_list, deaths_list, recovered_list, current_infected_list = [],[],[],[],[]
date_list = []
savedMonth = 0
confirmed, deaths, recovered = 0, 0, 0
for i in range(len(covid_19_data)):
    date = covid_19_data['ObservationDate'][i]
    inner_date_list = date.split('/')
    year = inner_date_list[2]
    month = inner_date_list[0]
    month = int(month)
    day = inner_date_list[1]
    day = int(day)
    confirmed += covid_19_data['Confirmed'][i]
    deaths += covid_19_data['Deaths'][i]
    recovered += covid_19_data['Recovered'][i]
    if savedMonth == month:
        if day % 5 == 0:
            indiDate = str(day)
        else:
            indiDate = ''
    else:
        savedMonth = month
        indiDate = str(month)+' / '+str(day)
    if i == 0 or date_list[-1] != date:
        date_list.append(date) 
        indi_date_list.append(indiDate)
        
    if (i < len(covid_19_data) - 1 and covid_19_data['ObservationDate'][i + 1] != date) or i == len(covid_19_data) - 1:
        current_infected = confirmed - deaths - recovered
        confirmed_list.append(confirmed)
        deaths_list.append(deaths)
        recovered_list.append(recovered)
        current_infected_list.append(current_infected)
        confirmed, deaths, recovered = 0, 0, 0
        
# Display

import matplotlib.pyplot as plt
plt.figure(figsize=(10, 7), dpi=100)
plt.title('Global')
plt.xlabel('Date')
plt.ylabel('Number of people')
plt.plot(confirmed_list, label='Confirmed', color='purple')
plt.plot(deaths_list, label='Deaths', color='r')
plt.plot(recovered_list, label='Recovered', color='b')
plt.bar(range(len(indi_date_list)), current_infected_list, label='Active', color='y')
x = np.arange(len(indi_date_list))
plt.xticks(x[:], indi_date_list[:], rotation=-90)
plt.legend()
plt.grid(True)
plt.show()

print('{}/{}/{}'.format(year, month, day))
print('Confirmed:{}'.format(int(confirmed_list[-1])))
print('Deaths:{}'.format(int(deaths_list[-1])))
print('Recovered:{}'.format(int(recovered_list[-1])))
print('Active:{}'.format(int(current_infected_list[-1])))

## 1-2. Prediction using regression analysis

In [ ]:
# max dimension for regression analysis
max_dimension = 7

# predict days
predict_days = 180

In [ ]:
# Format data

train_list = []
confirmed, deaths, recovered = 0, 0, 0
for i in range(len(covid_19_data)):
    date = covid_19_data['ObservationDate'][i]
    inner_date_list = date.split('/')
    y = inner_date_list[2]
    m = inner_date_list[0]
    d = inner_date_list[1]
    confirmed += int(covid_19_data['Confirmed'][i])
    deaths += int(covid_19_data['Deaths'][i])
    recovered += int(covid_19_data['Recovered'][i])
    if i == 0:
        saved_date = d
    if i == len(covid_19_data) - 1:
        next_d = ''
    else:
        next_date = covid_19_data['ObservationDate'][i + 1]
        next_date_list = next_date.split('/')
        next_y = next_date_list[2]
        next_m = next_date_list[0]
        next_d = next_date_list[1]
    if next_y != y or next_m != m or next_d != d:
        current_infected = confirmed - deaths - recovered
        inner_dic = {'date':date,
#                     'num_days':i,
                     'confirmed':confirmed,
                     'deaths':deaths,
                     'recovered':recovered,
                     'current_infected':current_infected}
        train_list.append(inner_dic)
        saved_date = date
        confirmed, deaths, recovered = 0, 0, 0

train_list = pd.DataFrame(train_list)
train_list

In [ ]:
# Preprocessing

import datetime

last_date = train_list['date'][len(train_list) - 1]
inner_date_list = last_date.split('/')
year = int(inner_date_list[2])
month = int(inner_date_list[0])
day = int(inner_date_list[1])
dt_last = datetime.date(year, month, day)
add_date_list = []
for i in range(predict_days):
    new_date = dt_last + datetime.timedelta(days = i + 1)
    y, m, d = new_date.year, new_date.month, new_date.day
    if m < 10:
        m = '0' + str(m)
    if d < 10:
        d = '0' + str(d)
    new_date = str(m)+'/'+str(d)+'/'+str(y)
    add_date_list.append(new_date)
add_date_list = np.array(add_date_list)

# Nonlinear regression analysis

import matplotlib.dates as mdates

fig = plt.figure()
ax = fig.add_subplot(1,1,1)

x = np.array(train_list['date'])
y = np.array(train_list['confirmed'])
x1 = np.arange(len(x))

score_list = []
for j in range(1, max_dimension):
    #**********************************************************
    fit = np.polyfit(x1,y,j)
    #**********************************************************
    y2 = np.poly1d(fit)(x1)
    # r2_score
    from sklearn.metrics import r2_score
    score = r2_score(y,y2)
    score_list.append(score)
max_score = max(score_list)
max_index = score_list.index(max(score_list))
fit = np.polyfit(x1,y,max_index)
y2 = np.poly1d(fit)(x1)

# predict
temp_date = np.append(x, add_date_list)
x2 = x
predict_list = []
for i in range(len(x) - 1, len(temp_date)):
    predict_y = np.poly1d(fit)(i)
    if predict_y >= 0:
        x2 = np.append(x2, temp_date[i])
        predict_list.append(predict_y)
    else:
        break
predict_list = np.array(predict_list)
y3 = np.append(y2, predict_list)

ax.plot(x,y,'bo', color='y') 
ax.plot(x2,y3,'--k', color='g') 

plt.title('Confirmed')
plt.xlabel("Date")
plt.ylabel("Number of people")
plt.xticks(np.arange(0, len(x2), 10), rotation=-90)
plt.grid(True)
plt.tight_layout()
plt.show()

# Expected convergence date
if i >= len(x) + predict_days - 1:
    conv_date = 'Unknown'
else:
    conv_date = x2[-1]
print('Expected convergence date : {}'.format(conv_date))
print('Score:{:.4f}'.format(max_score))
print('Dimension:{}'.format(max_index))

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)

x = np.array(train_list['date'])
y = np.array(train_list['deaths'])
x1 = np.arange(len(x))


score_list = []
for j in range(1, max_dimension):
    #**********************************************************
    fit = np.polyfit(x1,y,j)
    #**********************************************************
    y2 = np.poly1d(fit)(x1)
    # r2_score
    from sklearn.metrics import r2_score
    score = r2_score(y,y2)
    score_list.append(score)
max_score = max(score_list)
max_index = score_list.index(max(score_list))
fit = np.polyfit(x1,y,max_index)
y2 = np.poly1d(fit)(x1)


# predict
temp_date = np.append(x, add_date_list)
x2 = x
predict_list = []
for i in range(len(x) - 1, len(temp_date)):
    predict_y = np.poly1d(fit)(i)
    if predict_y >= 0:
        x2 = np.append(x2, temp_date[i])
        predict_list.append(predict_y)
    else:
        break
predict_list = np.array(predict_list)
y3 = np.append(y2, predict_list)

ax.plot(x,y,'bo', color='y') 
ax.plot(x2,y3,'--k', color='g') 

plt.title('Deaths')
plt.xlabel("Date")
plt.ylabel("Number of people")
plt.xticks(np.arange(0, len(x2), 10), rotation=-90)
plt.grid(True)
plt.tight_layout()
plt.show()

# Expected convergence date
if i >= len(x) + predict_days - 1:
    conv_date = 'Unknown'
else:
    conv_date = x2[-1]
print('Expected convergence date : {}'.format(conv_date))
print('Score:{:.4f}'.format(max_score))
print('Dimension:{}'.format(max_index))

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)

x = np.array(train_list['date'])
y = np.array(train_list['recovered'])
x1 = np.arange(len(x))


score_list = []
for j in range(1, max_dimension):
    #**********************************************************
    fit = np.polyfit(x1,y,j)
    #**********************************************************
    y2 = np.poly1d(fit)(x1)
    # r2_score
    from sklearn.metrics import r2_score
    score = r2_score(y,y2)
    score_list.append(score)
max_score = max(score_list)
max_index = score_list.index(max(score_list))
fit = np.polyfit(x1,y,max_index)
y2 = np.poly1d(fit)(x1)


# predict
temp_date = np.append(x, add_date_list)
x2 = x
predict_list = []
for i in range(len(x) - 1, len(temp_date)):
    predict_y = np.poly1d(fit)(i)
    if predict_y >= 0:
        x2 = np.append(x2, temp_date[i])
        predict_list.append(predict_y)
    else:
        break
predict_list = np.array(predict_list)
y3 = np.append(y2, predict_list)

ax.plot(x,y,'bo', color='y') 
ax.plot(x2,y3,'--k', color='g') 

plt.title('Recovered')
plt.xlabel("Date")
plt.ylabel("Number of people")
plt.xticks(np.arange(0, len(x2), 10), rotation=-90)
plt.grid(True)
plt.tight_layout()
plt.show()

# Expected convergence date
if i >= len(x) + predict_days - 1:
    conv_date = 'Unknown'
else:
    conv_date = x2[-1]
print('Expected convergence date : {}'.format(conv_date))
print('Score:{:.4f}'.format(max_score))
print('Dimension:{}'.format(max_index))

In [ ]:
#import matplotlib.dates as mdates

fig = plt.figure()
ax = fig.add_subplot(1,1,1)

x = np.array(train_list['date'])
y = np.array(train_list['current_infected'])
x1 = np.arange(len(x))


score_list = []
for j in range(1, max_dimension):
    #**********************************************************
    fit = np.polyfit(x1,y,j)
    #**********************************************************
    y2 = np.poly1d(fit)(x1)
    # r2_score
    from sklearn.metrics import r2_score
    score = r2_score(y,y2)
    score_list.append(score)
max_score = max(score_list)
max_index = score_list.index(max(score_list))
fit = np.polyfit(x1,y,max_index)
y2 = np.poly1d(fit)(x1)


# predict
temp_date = np.append(x, add_date_list)
x2 = x
predict_list = []
for i in range(len(x) - 1, len(temp_date)):
    predict_y = np.poly1d(fit)(i)
    if predict_y >= 0:
        x2 = np.append(x2, temp_date[i])
        predict_list.append(predict_y)
    else:
        break
predict_list = np.array(predict_list)
y3 = np.append(y2, predict_list)

ax.plot(x,y,'bo', color='y') 
ax.plot(x2,y3,'--k', color='g') 

plt.title('Active')
plt.xlabel("Date")
plt.ylabel("Number of people")
plt.xticks(np.arange(0, len(x2), 10), rotation=-90)
plt.grid(True)
plt.tight_layout()
plt.show()

# Expected convergence date
if i >= len(x) + predict_days - 1:
    conv_date = 'Unknown'
else:
    conv_date = x2[-1]
print('Expected convergence date : {}'.format(conv_date))
print('Score:{:.4f}'.format(max_score))
print('Dimension:{}'.format(max_index))

# 2. For each country
## Specify country name
### Default country name is Japan
* Replacing "Japan" with the name of another country will be result for there.
* You can use the country names shown in the list below.

### Make lists of country names
* ↓ Click "Show hidden cell" button to display the list

In [ ]:
country_name_list = []
inner_name = ''
for i in range(len(covid_19_data)):
    inner_name = covid_19_data['Country/Region'][i]
    flg = 0
    for j in range(len(country_name_list)):
        if country_name_list[j] == inner_name:
            flg = 1
            break
    if flg == 0:
        country_name_list.append(inner_name)
country_name_list.sort()
for i in range(len(country_name_list)):
    print(country_name_list[i])

In [ ]:
# Go to "Edit" 

# And Change here ↓ (For example... country_name = 'US')
country_name = 'Japan'

# And "Run All" (No accelerator needed)

## 2-1. Overview

In [ ]:
# Cut out own country's data

covid_19_own = []
for i in range(len(covid_19_data)):
    if covid_19_data['Country/Region'][i] == country_name:
        covid_19_own.append(covid_19_data.loc[i])
covid_19_own = pd.DataFrame(covid_19_own).reset_index()
covid_19_own

In [ ]:
# Format and extract items

indi_date_list, confirmed_list, deaths_list, recovered_list, current_infected_list = [],[],[],[],[]
date_list = []
savedMonth = 0
confirmed, deaths, recovered = 0, 0, 0
for i in range(len(covid_19_own)):
    date = covid_19_own['ObservationDate'][i]
    inner_date_list = date.split('/')
    year = inner_date_list[2]
    month = inner_date_list[0]
    month = int(month)
    day = inner_date_list[1]
    day = int(day)
    confirmed += covid_19_own['Confirmed'][i]
    deaths += covid_19_own['Deaths'][i]
    recovered += covid_19_own['Recovered'][i]
    if savedMonth == month:
        if day % 5 == 0:
            indiDate = str(day)
        else:
            indiDate = ''
    else:
        savedMonth = month
        indiDate = str(month)+' / '+str(day)
    if i == 0 or date_list[-1] != date:
        date_list.append(date) 
        indi_date_list.append(indiDate)
        
    if (i < len(covid_19_own) - 1 and covid_19_own['ObservationDate'][i + 1] != date) or i == len(covid_19_own) - 1:
        current_infected = confirmed - deaths - recovered
        confirmed_list.append(confirmed)
        deaths_list.append(deaths)
        recovered_list.append(recovered)
        current_infected_list.append(current_infected)
        confirmed, deaths, recovered = 0, 0, 0
#print(int(confirmed_list[-1]))
#print(int(deaths_list[-1]))
#print(int(recovered_list[-1]))

In [ ]:
# Display

import matplotlib.pyplot as plt
plt.figure(figsize=(10, 7), dpi=100)
plt.title('Covid 19 infection status in ' + country_name)
plt.xlabel('Date')
plt.ylabel('Number of people')
plt.plot(confirmed_list, label='Confirmed')
plt.plot(deaths_list, label='Deaths')
plt.plot(recovered_list, label='Recovered')
plt.bar(range(len(indi_date_list)), current_infected_list, label='Active', color='pink')
x = np.arange(len(indi_date_list))
plt.xticks(x[:], indi_date_list[:], rotation=-90)
plt.legend()
plt.grid(True)
plt.show()

print('{}/{}/{}'.format(year, month, day))
print('Confirmed:{}'.format(int(confirmed_list[-1])))
print('Deaths:{}'.format(int(deaths_list[-1])))
print('Recovered:{}'.format(int(recovered_list[-1])))
print('Active:{}'.format(int(current_infected_list[-1])))

## 2-2. Prediction using regression analysis

In [ ]:
# Format data

train_list = []
confirmed, deaths, recovered = 0, 0, 0
for i in range(len(covid_19_own)):
    date = covid_19_own['ObservationDate'][i]
    inner_date_list = date.split('/')
    y = inner_date_list[2]
    m = inner_date_list[0]
    d = inner_date_list[1]
    confirmed += int(covid_19_own['Confirmed'][i])
    deaths += int(covid_19_own['Deaths'][i])
    recovered += int(covid_19_own['Recovered'][i])
    if i == 0:
        saved_date = d
    if i == len(covid_19_own) - 1:
        next_d = ''
    else:
        next_date = covid_19_own['ObservationDate'][i + 1]
        next_date_list = next_date.split('/')
        next_y = next_date_list[2]
        next_m = next_date_list[0]
        next_d = next_date_list[1]
    if next_y != y or next_m != m or next_d != d:
        current_infected = confirmed - deaths - recovered
        inner_dic = {'date':date,
#                     'num_days':i,
                     'confirmed':confirmed,
                     'deaths':deaths,
                     'recovered':recovered,
                     'current_infected':current_infected}
        train_list.append(inner_dic)
        saved_date = date
        confirmed, deaths, recovered = 0, 0, 0

train_list = pd.DataFrame(train_list)
train_list

In [ ]:
# Preprocessing

last_date = train_list['date'][len(train_list) - 1]
inner_date_list = last_date.split('/')
year = int(inner_date_list[2])
month = int(inner_date_list[0])
day = int(inner_date_list[1])
dt_last = datetime.date(year, month, day)
add_date_list = []
for i in range(predict_days):
    new_date = dt_last + datetime.timedelta(days = i + 1)
    y, m, d = new_date.year, new_date.month, new_date.day
    if m < 10:
        m = '0' + str(m)
    if d < 10:
        d = '0' + str(d)
    new_date = str(m)+'/'+str(d)+'/'+str(y)
    add_date_list.append(new_date)
add_date_list = np.array(add_date_list)

# Nonlinear regression analysis

import matplotlib.dates as mdates

fig = plt.figure()
ax = fig.add_subplot(1,1,1)

x = np.array(train_list['date'])
y = np.array(train_list['confirmed'])
x1 = np.arange(len(x))

score_list = []
for j in range(1, max_dimension):
    #**********************************************************
    fit = np.polyfit(x1,y,j)
    #**********************************************************
    y2 = np.poly1d(fit)(x1)
    # r2_score
    from sklearn.metrics import r2_score
    score = r2_score(y,y2)
    score_list.append(score)
max_score = max(score_list)
max_index = score_list.index(max(score_list))
fit = np.polyfit(x1,y,max_index)
y2 = np.poly1d(fit)(x1)

# predict
temp_date = np.append(x, add_date_list)
x2 = x
predict_list = []
for i in range(len(x) - 1, len(temp_date)):
    predict_y = np.poly1d(fit)(i)
    if predict_y >= 0:
        x2 = np.append(x2, temp_date[i])
        predict_list.append(predict_y)
    else:
        break
predict_list = np.array(predict_list)
y3 = np.append(y2, predict_list)

ax.plot(x,y,'bo', color='pink') 
ax.plot(x2,y3,'--k', color='g') 

plt.title('Confirmed')
plt.xlabel("Date")
plt.ylabel("Number of people")
plt.xticks(np.arange(0, len(x2), 10), rotation=-90)
plt.grid(True)
plt.tight_layout()
plt.show()

# Expected convergence date
if i >= len(x) + predict_days - 1:
    conv_date = 'Unknown'
else:
    conv_date = x2[-1]
print('Expected convergence date : {}'.format(conv_date))
print('Score:{:.4f}'.format(max_score))
print('Dimension:{}'.format(max_index))

In [ ]:

fig = plt.figure()
ax = fig.add_subplot(1,1,1)

x = np.array(train_list['date'])
y = np.array(train_list['deaths'])
x1 = np.arange(len(x))


score_list = []
for j in range(1, max_dimension):
    #**********************************************************
    fit = np.polyfit(x1,y,j)
    #**********************************************************
    y2 = np.poly1d(fit)(x1)
    # r2_score
    from sklearn.metrics import r2_score
    score = r2_score(y,y2)
    score_list.append(score)
max_score = max(score_list)
max_index = score_list.index(max(score_list))
fit = np.polyfit(x1,y,max_index)
y2 = np.poly1d(fit)(x1)


# predict
temp_date = np.append(x, add_date_list)
x2 = x
predict_list = []
for i in range(len(x) - 1, len(temp_date)):
    predict_y = np.poly1d(fit)(i)
    if predict_y >= 0:
        x2 = np.append(x2, temp_date[i])
        predict_list.append(predict_y)
    else:
        break
predict_list = np.array(predict_list)
y3 = np.append(y2, predict_list)

ax.plot(x,y,'bo', color='pink') 
ax.plot(x2,y3,'--k', color='g') 

plt.title('Deaths')
plt.xlabel("Date")
plt.ylabel("Number of people")
plt.xticks(np.arange(0, len(x2), 10), rotation=-90)
plt.grid(True)
plt.tight_layout()
plt.show()

# Expected convergence date
if i >= len(x) + predict_days - 1:
    conv_date = 'Unknown'
else:
    conv_date = x2[-1]
print('Expected convergence date : {}'.format(conv_date))
print('Score:{:.4f}'.format(max_score))
print('Dimension:{}'.format(max_index))

In [ ]:

fig = plt.figure()
ax = fig.add_subplot(1,1,1)

x = np.array(train_list['date'])
y = np.array(train_list['recovered'])
x1 = np.arange(len(x))


score_list = []
for j in range(1, max_dimension):
    #**********************************************************
    fit = np.polyfit(x1,y,j)
    #**********************************************************
    y2 = np.poly1d(fit)(x1)
    # r2_score
    from sklearn.metrics import r2_score
    score = r2_score(y,y2)
    score_list.append(score)
max_score = max(score_list)
max_index = score_list.index(max(score_list))
fit = np.polyfit(x1,y,max_index)
y2 = np.poly1d(fit)(x1)


# predict
temp_date = np.append(x, add_date_list)
x2 = x
predict_list = []
for i in range(len(x) - 1, len(temp_date)):
    predict_y = np.poly1d(fit)(i)
    if predict_y >= 0:
        x2 = np.append(x2, temp_date[i])
        predict_list.append(predict_y)
    else:
        break
predict_list = np.array(predict_list)
y3 = np.append(y2, predict_list)

ax.plot(x,y,'bo', color='pink') 
ax.plot(x2,y3,'--k', color='g') 

plt.title('Recovered')
plt.xlabel("Date")
plt.ylabel("Number of people")
plt.xticks(np.arange(0, len(x2), 10), rotation=-90)
plt.grid(True)
plt.tight_layout()
plt.show()

# Expected convergence date
if i >= len(x) + predict_days - 1:
    conv_date = 'Unknown'
else:
    conv_date = x2[-1]
print('Expected convergence date : {}'.format(conv_date))
print('Score:{:.4f}'.format(max_score))
print('Dimension:{}'.format(max_index))

In [ ]:

fig = plt.figure()
ax = fig.add_subplot(1,1,1)

x = np.array(train_list['date'])
y = np.array(train_list['current_infected'])
x1 = np.arange(len(x))


score_list = []
for j in range(1, max_dimension):
    #**********************************************************
    fit = np.polyfit(x1,y,j)
    #**********************************************************
    y2 = np.poly1d(fit)(x1)
    # r2_score
    from sklearn.metrics import r2_score
    score = r2_score(y,y2)
    score_list.append(score)
max_score = max(score_list)
max_index = score_list.index(max(score_list))
fit = np.polyfit(x1,y,max_index)
y2 = np.poly1d(fit)(x1)


# predict
temp_date = np.append(x, add_date_list)
x2 = x
predict_list = []
for i in range(len(x) - 1, len(temp_date)):
    predict_y = np.poly1d(fit)(i)
    if predict_y >= 0:
        x2 = np.append(x2, temp_date[i])
        predict_list.append(predict_y)
    else:
        break
predict_list = np.array(predict_list)
y3 = np.append(y2, predict_list)

ax.plot(x,y,'bo', color='pink') 
ax.plot(x2,y3,'--k', color='g') 

plt.title('Active')
plt.xlabel("Date")
plt.ylabel("Number of people")
plt.xticks(np.arange(0, len(x2), 10), rotation=-90)
plt.grid(True)
plt.tight_layout()
plt.show()

# Expected convergence date
if i >= len(x) + predict_days - 1:
    conv_date = 'Unknown'
else:
    conv_date = x2[-1]
print('Expected convergence date : {}'.format(conv_date))
print('Score:{:.4f}'.format(max_score))
print('Dimension:{}'.format(max_index))